![banner](../images/banner.png)

Este é um estudo que tem como objetivo entender o cenário que envolve doenças respiratórias virais e a mortalidade no Brasil. Para isso, foram utilizados dados do Sistema de Informação de Vigilância Epidemiológica da Gripe (SIVEP-Gripe) do Ministério da Saúde. Os dados foram coletados entre o início de Janeiro e o final de Agosto de 2023.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Lendo os dados
data = pd.read_csv("../data/raw/INFLUD23-16-10-2023.csv", sep=";", low_memory=False)

In [3]:
# Verificando as dimensões do dataset
print("Dimensões do dataset:\n")
print("Linhas:\t\t{}".format(data.shape[0]))
print("Colunas:\t{}".format(data.shape[1]))

Dimensões do dataset:

Linhas:		226763
Colunas:	173


# Perguntas de interesse:
1. Quais as doenças respiratórias mais comuns?
2. Qual o tempo médio desde a apresentação dos sintomas até o resultado final?
3. Qual a porcentagem de pacientes que foram internados? Quantos foram a óbito? Quantos tinham comorbidades?
4. Qual a idade média dos pacientes que foram à óbito?
5. Quais as doenças mais comum entre os pacientes que foram à óbito?

## 1. Quais as doenças respiratórias mais comuns?

Como primeiro passo, vamos buscar entender quais as 10 doenças respiratórias mais comuns entre os pacientes que foram atendidos no período de Janeiro a Agosto de 2023. Para isso, vamos utilizar a coluna `CLASSI_FIN`, que contém a classificação final do caso, e a coluna `CLASSI_OUT`, que contém a classificação final do caso após investigação.

In [4]:
# Alterando os valores numéricos para os nomes das categorias
dict_novas_cat = {1.0:"Influenza",
                  2.0:"Outro vírus",
                  3.0:3.0,
                  4.0:"Não especificado",
                  5.0:"Covid-19"}

# Aplicando a mudança
data["CLASSI_FIN"] = data["CLASSI_FIN"].map(dict_novas_cat)

# Buscando as classes complementares
data.loc[data["CLASSI_FIN"] == 3, "CLASSI_FIN"] = data.loc[data["CLASSI_FIN"] == 3, "CLASSI_OUT"]

In [5]:
# Buscando as dez doenças mais comuns
df_doencas_mais_comuns = data.CLASSI_FIN.value_counts()[0:10].to_frame()

# Tratando o index e o nome das colunas
df_doencas_mais_comuns = df_doencas_mais_comuns.reset_index()
df_doencas_mais_comuns = df_doencas_mais_comuns.rename(columns={"CLASSI_FIN":"Doenças", 
                                                                "count":"Quantidade"})

In [87]:
# Adicionando a figure
fig = go.Figure()

# Criando o gráfico de barras
fig.add_trace(go.Bar(x=df_doencas_mais_comuns["Doenças"],
                     y=df_doencas_mais_comuns["Quantidade"],
                     text=df_doencas_mais_comuns["Quantidade"],
                     textposition='outside'))

# Atualizando os traços
fig.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False,
                  marker_color=['crimson', '#555655', '#555655', '#555655', 
                                '#555655', '#555655', '#555655', '#555655',
                                '#555655', '#555655'])

# Atualizando o layout
fig.update_layout(
    title='<b>Os 10 Principais Agentes Causadores de SRAGS no Brasil em 2023</b><br><sup><i>Em sua maioria, os casos não possuem uma conclusão sobre o agente causador da doença.</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    bargap=0.20,
    bargroupgap=0.1,
    template='plotly_white',
    legend=dict(yanchor="top", y=0.90, xanchor="left", x=0.01, title=None),
    yaxis=dict(showticklabels=False, showgrid=False),
    xaxis=dict(showgrid=False),
    title_x=0.1,
    width=910,
    height=500
)

# Adicionando o texto
fig.add_annotation(x=2.96, y=85000,
                   text="Aproximadamente <span style='color: crimson;'><b>52%</b></span> dos casos de SRAGS não possuem uma <br>conclusão sobre o agente causador da doença.",
                   showarrow=False,
                   yshift=50)

# Adicionando a linha
fig.add_shape(type="line",
    x0=0.65, y0=116000, x1=5.25, y1=116000,
    line=dict(color="crimson",width=3)
)

## 2. Qual o tempo médio desde a apresentação dos sintomas até o resultado final?
Agora, vamos entender qual o tempo médio para ober o diagnóstico final desde a apresentação dos sintomas.

In [7]:
# Alterando o formato das datas
data['DT_SIN_PRI'] = pd.to_datetime(data['DT_SIN_PRI'], format='%d/%m/%Y')
data['DT_RES'] = pd.to_datetime(data['DT_RES'], format='%d/%m/%Y')

In [8]:
# Calculando o tempo decorrido
df_tempo_decorrido = data['DT_RES'] - data['DT_SIN_PRI']

In [9]:
# Calculando a porcentagem de valores nulos
print(f"{round(((df_tempo_decorrido.isna().sum() / df_tempo_decorrido.shape[0]) * 100), 2)}% dos pacientes não obtiveram um resultado sobre a doença.")

98.36% dos pacientes não obtiveram um resultado sobre a doença.


In [10]:
# Obtendo os dias em formato numérico
df_tempo_decorrido = df_tempo_decorrido.dt.days.to_frame()
df_tempo_decorrido = df_tempo_decorrido.rename(columns={0:"Dias"})

In [11]:
# Gerando uma figure
fig = go.Figure()

# Adicionando o indicador
fig.add_trace(go.Indicator(
    mode = "number",
    value = df_tempo_decorrido["Dias"].mean(),
    title = {"text": "Média de tempo para o diagnóstico final"},
    delta = None,
    number = {'suffix': " dias", 'font': {'color': 'crimson'}}))

## 3. Qual a porcentagem de pacientes que foram internados? Quantos foram a óbito? Quantos tinham comorbidades?

Nesta etapa, vamos buscar entender qual a porcentagem de pacientes que foram 
internados e quantos foram à óbito. Antes disso, teremos que tratar os dados para
que possamos obter a informação correta.

Onde: <br>

`EVOLUCAO` = 1 - Cura <br>
`EVOLUCAO` = 2 - Óbito <br>
`EVOLUCAO` = 3 - Óbito por outras causas <br>
`EVOLUCAO` = 9 - Ignorado <br>
<br>
`HOSPITAL` = 1 - Sim <br>
`HOSPITAL` = 2 - Não <br>
`HOSPITAL` = 9 - Ignorado <br>
<br>
`FATOR_RISC` = 1 - Sim <br>
`FATOR_RISC` = 2 - Não <br>
`FATOR_RISC` = 9 - Ignorado <br>

In [12]:
# Alterando os rótulos
data['HOSPITAL'] = data['HOSPITAL'].map({1.0:"Sim", 2.0:"Não", 9.0:"Não Informado"})
data['EVOLUCAO'] = data['EVOLUCAO'].map({1.0:"Cura", 2.0:"Óbito", 3.0:"Óbito", 9.0:"Ignorado"})
data['FATOR_RISC'] = data['FATOR_RISC'].map({1.0:"Sim", 2.0:"Não", 9.0:"Não Informado"})

In [13]:
# Obtendo os dados
df_situacao_pacientes = data[["HOSPITAL", "EVOLUCAO", "FATOR_RISC"]].value_counts().to_frame().reset_index()
df_situacao_pacientes = df_situacao_pacientes.rename(columns={"EVOLUCAO":"SITUACAO",
                                                              "HOSPITAL":"INTERNADO",
                                                              "count":"QUANTIDADE"})

In [90]:
# Calculando as porcentagens
prcnt_internados = round(df_situacao_pacientes.query("INTERNADO == 'Sim'")['QUANTIDADE'].sum() / data.shape[0] * 100, 2)
prcnt_internados_obito = round(df_situacao_pacientes.query("INTERNADO == 'Sim' and SITUACAO == 'Óbito'")['QUANTIDADE'].sum() / data.shape[0] * 100, 2)
prcnt_internados_obito_comorbidade = round(df_situacao_pacientes.query("INTERNADO == 'Sim'and SITUACAO == 'Óbito' and FATOR_RISC == 'Sim'")['QUANTIDADE'].sum() / data.shape[0] * 100, 2)

# Adicionando a figure
fig = go.Figure()

fig.add_trace(go.Funnelarea(text=["<b>100%</b> <br>", 
                                  f"<b>{prcnt_internados}%</b>", 
                                  f"<b>{prcnt_internados_obito}%</b>",
                                  f"<b>{prcnt_internados_obito_comorbidade}%</b>"],
                            values=[data.shape[0], 
                                    df_situacao_pacientes.query("INTERNADO == 'Sim'")["QUANTIDADE"].sum(), 
                                    df_situacao_pacientes.query("SITUACAO == 'Óbito'")["QUANTIDADE"].sum(),
                                    df_situacao_pacientes.query("SITUACAO == 'Óbito' and FATOR_RISC == 'Sim'")["QUANTIDADE"].sum()],
                            textposition="inside",
                            textinfo="text",
                            textfont={"size": 16},
                            marker={"colors":["#f1666d", "#dc143c", "#9c0720", "#c57d56"],
                                    "line":{"color":"white", "width":10}}))

# Atualizando o layout da figure
fig.update_layout(
    title='<b>Internações e Óbitos de Pacientes com SRAGS</b><br><sup><i>A maior parte dos óbitos são de pessoas com comorbidades.</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=False, showgrid=False),
    xaxis=dict(showgrid=False),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=False
)

# Adicionando as anotações
fig.add_annotation(x=0.12, y=0.64,
                   text="<span style='color: #646369;'><b>Pacientes</b></span>",
                   showarrow=False,
                   yshift=50)

fig.add_annotation(x=0.12, y=0.20,
                   text="<span style='color: #646369;'><b>Internados</b></span>",
                   showarrow=False,
                   yshift=50)

fig.add_annotation(x=0.12, y=-0.07,
                   text="<span style='color: #646369;'><b>Óbitos</b></span>",
                   showarrow=False,
                   yshift=50)

fig.add_annotation(x=0.12, y=-0.15,
                   text="<span style='color: #646369;'><b>Comorbidade</b></span>",
                   showarrow=False,
                   yshift=50)

fig.add_annotation(x=0.97, y=0.03,
                   text="Apesar do alto número de <span style='color: crimson;'><b>Internados</b></span>,<br>podemos observar que o número de <span style='color: crimson;'><br><b>Óbitos</b></span> é relativamente baixo, sendo<br>em sua maioria pessoas com<br><span style='color: crimson;'><b>Comorbidades</b></span>.",
                   showarrow=False,
                   yshift=50)

# Adicionando a linha 
fig.add_shape(type="line",
    x0=0.69, y0=0.447, x1=0.97, y1=0.447,
    line=dict(color="crimson",width=3))

## 4. Qual a idade média dos pacientes que foram a óbito?

Nessa etapa, vamos explorar a variável ```NU_IDADE_N```, que não possui valores 
nulos e representa a idade do paciente.

In [28]:
# Gerando uma figure
fig = go.Figure()

# Adicionando o indicador
fig.add_trace(go.Indicator(
    mode = "number",
    value = data.query("EVOLUCAO == 'Óbito'")["NU_IDADE_N"].mean(),
    title = {"text": "Idade média dos pacientes que foram a óbito"},
    delta = None,
    number = {'suffix': " anos", 'font': {'color': 'crimson'}}))

## 5. Qual doença mais comum entre os pacientes que foram à óbito?

Na última fase, vamos repetir o processo da primeira etapa, mas dessa vez vamos
explorar apenas os pacientes que foram à óbito. Como já corrigimos os valores
da coluna `CLASSI_FIN` na primeira etapa, não será necessário corrigir novamente.

In [33]:
# Buscando as dez doenças mais comuns
df_doencas_mais_comuns_obito = data.query("EVOLUCAO == 'Óbito'")["CLASSI_FIN"].value_counts()[0:10].to_frame() 

# Tratando o index e o nome das colunas
df_doencas_mais_comuns_obito = df_doencas_mais_comuns_obito.reset_index()
df_doencas_mais_comuns_obito = df_doencas_mais_comuns_obito.rename(columns={"CLASSI_FIN":"Doenças", 
                                                                "count":"Quantidade"})

In [83]:
# Adicionando a figure
fig = go.Figure()

# Criando o gráfico de barras
fig.add_trace(go.Bar(x=df_doencas_mais_comuns_obito["Doenças"],
                     y=df_doencas_mais_comuns_obito["Quantidade"],
                     text=df_doencas_mais_comuns_obito["Quantidade"],
                     textposition='outside'))

# Atualizando os traços
fig.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False,
                  marker_color=['#555655', 'crimson', '#555655', '#555655', 
                                '#555655', '#555655', '#555655', '#555655',
                                '#555655', '#555655'])

# Atualizando o layout
fig.update_layout(
    title='<b>Os 10 Principais Agentes Causadores de Óbitos em 2023</b><br><sup><i>Ignorando os casos não identificados, que são maioria, o principal agente causador de mortes foi a Covid-19.</i></sup>',
    title_font=dict(size=23, family="Arial"),
    font_color='#646369',
    bargap=0.20,
    bargroupgap=0.1,
    template='plotly_white',
    legend=dict(yanchor="top", y=0.90, xanchor="left", x=0.01, title=None),
    yaxis=dict(showticklabels=False, showgrid=False),
    xaxis=dict(showgrid=False),
    title_x=0.1,
    width=910,
    height=500
)

# Adicionando o texto
fig.add_annotation(x=4.10, y=8000,
                   text="Mesmo após o fim da pandemia, a <span style='color: crimson;'><b>Covid-19</b></span> continua sendo o<br>vírus que mais causa mortes no Brasil, <span style='color: crimson;'><b>5.68 vezes</b></span> mais que a<br>Influenza.",
                   showarrow=False,
                   yshift=50)

# Adicionando a linha
fig.add_shape(type="line",
    x0=1.85, y0=11700, x1=6.35, y1=11700,
    line=dict(color="crimson",width=3)
)